# Bab 11 - Melatih Deep Neural Networks

**Tujuan Pembelajaran:**
Memperdalam pemahaman dan keterampilan praktis dalam mengimplementasikan konsep inti Machine Learning melalui reproduksi kode dan penjelasan teoritis yang terstruktur, menggunakan buku *Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow: Concepts, Tools, and Techniques to Build Intelligent Systems* (O’Reilly) sebagai referensi utama.

## Pendahuluan

Bab ini membahas tantangan-tantangan dalam melatih Jaringan Neural Dalam (DNN) dan teknik-teknik untuk mengatasinya. Kita akan menjelajahi masalah *vanishing/exploding gradients*, strategi inisialisasi yang baik, fungsi aktivasi non-saturasi, Batch Normalization, dan Gradient Clipping. Selanjutnya, kita akan mempelajari teknik *transfer learning* dan *unsupervised pretraining* untuk skenario data terbatas. Terakhir, kita akan melihat berbagai *optimizer* dan teknik regularisasi untuk meningkatkan kinerja dan stabilitas model.

---

## Bagian 1: Masalah Vanishing/Exploding Gradients

### 1.1. Latar Belakang Masalah

Saat melatih jaringan neural yang dalam, gradien seringkali menjadi sangat kecil ("vanishing gradients") atau sangat besar ("exploding gradients") seiring dengan propagasi mundur melalui lapisan-lapisan.

* **Vanishing Gradients**: Menyebabkan pembaruan bobot pada lapisan bawah menjadi sangat kecil, sehingga pelatihan tidak konvergen.
* **Exploding Gradients**: Menyebabkan pembaruan bobot menjadi sangat besar, sehingga algoritma menjadi tidak stabil dan menyimpang.

Kedua masalah ini membuat lapisan-lapisan yang lebih rendah sulit untuk dilatih secara efektif, menyebabkan pelatihan yang lambat atau tidak stabil.

### 1.2. Inisialisasi Glorot dan He

Masalah gradien yang tidak stabil sebagian besar disebabkan oleh kombinasi fungsi aktivasi saturasi (seperti sigmoid atau tanh) dan skema inisialisasi bobot yang populer pada masa itu (distribusi normal dengan mean 0 dan standar deviasi 1).  Glorot dan Bengio mengusulkan bahwa agar sinyal mengalir dengan baik ke depan (saat membuat prediksi) dan ke belakang (saat *backpropagating gradients*), varians output setiap lapisan harus sama dengan varians inputnya, dan gradien harus memiliki varians yang sama sebelum dan sesudah mengalir melalui lapisan.

**Inisialisasi Glorot (Xavier Initialization)**:
Untuk fungsi aktivasi sigmoid atau tanh, bobot harus diinisialisasi secara acak dari distribusi normal dengan mean 0 dan varians $\sigma^2 = \frac{1}{\text{fan}_{avg}}$ atau dari distribusi uniform antara $-r$ dan $+r$ dengan $r = \sqrt{\frac{3}{\text{fan}_{avg}}}$, di mana $\text{fan}_{avg} = (\text{fan}_{in} + \text{fan}_{out}) / 2$.

**Inisialisasi He**:
Untuk fungsi aktivasi ReLU dan variannya, bobot harus diinisialisasi dari distribusi normal dengan mean 0 dan varians $\sigma^2 = \frac{2}{\text{fan}_{in}}$ atau dari distribusi uniform dengan $r = \sqrt{\frac{6}{\text{fan}_{in}}}$.

**Kode Implementasi (Inisialisasi)**

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

# Contoh penggunaan inisialisasi He pada lapisan Dense
# Untuk ReLU dan variannya
he_normal_init = keras.initializers.HeNormal()
keras.layers.Dense(10, activation="relu", kernel_initializer=he_normal_init)

# Untuk SELU, gunakan inisialisasi LeCun
lecun_normal_init = keras.initializers.LecunNormal()
keras.layers.Dense(10, activation="selu", kernel_initializer=lecun_normal_init)

# Jika ingin kustomisasi lebih lanjut (contoh VarianceScaling)
he_avg_init = keras.initializers.VarianceScaling(scale=2., mode='fan_avg', distribution='uniform')
keras.layers.Dense(10, activation="sigmoid", kernel_initializer=he_avg_init)

<Dense name=dense_2, built=False>

### 1.3. Fungsi Aktivasi Non-Saturasi

Fungsi aktivasi sigmoid dan tanh mengalami masalah saturasi, di mana gradien menjadi sangat dekat dengan nol untuk input yang besar (negatif atau positif), yang menyebabkan *vanishing gradients*.  Fungsi aktivasi non-saturasi membantu mengurangi masalah ini.

* **ReLU (Rectified Linear Unit)**: $ReLU(z) = max(0, z)$. Cepat dihitung dan tidak saturasi untuk nilai positif. Namun, dapat mengalami masalah "dying ReLUs" di mana neuron berhenti mengeluarkan apa pun selain 0.
* **Leaky ReLU**: $LeakyReLU_{\alpha}(z) = max(\alpha z, z)$. Memiliki sedikit kemiringan untuk $z < 0$ ($\alpha$ adalah hiperparameter, biasanya 0.01), yang mencegah neuron "mati".
* **PReLU (Parametric Leaky ReLU)**: Mirip dengan Leaky ReLU, tetapi $\alpha$ dipelajari selama pelatihan.
* **ELU (Exponential Linear Unit)**: $ELU_{\alpha}(z) = \alpha(exp(z) - 1)$ jika $z < 0$, dan $z$ jika $z \ge 0$. Mengungguli varian ReLU dalam eksperimen, mengurangi waktu pelatihan dan meningkatkan kinerja. Mengambil nilai negatif, yang membantu meredakan masalah *vanishing gradients*.
* **SELU (Scaled ELU)**: Varian ELU yang diskalakan. Jika jaringan hanya terdiri dari lapisan Dense dan menggunakan fungsi aktivasi SELU, serta bobot diinisialisasi dengan inisialisasi LeCun normal, jaringan akan *self-normalize*, yang secara efektif menyelesaikan masalah *vanishing/exploding gradients*.

In [2]:
# Contoh model dengan fungsi aktivasi ELU dan inisialisasi HeNormal
model_elu = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dense(10, activation="softmax")
])

# Contoh model dengan fungsi aktivasi Leaky ReLU
model_leaky_relu = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(alpha=0.2), # Tambahkan Leaky ReLU sebagai lapisan terpisah
    keras.layers.Dense(100, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(alpha=0.2),
    keras.layers.Dense(10, activation="softmax")
])

# Contoh model dengan fungsi aktivasi SELU (membutuhkan input yang distandardisasi)
# Pastikan X_train_scaled, X_valid_scaled, X_test_scaled sudah distandardisasi (mean 0, std 1)
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train.astype(np.float32).reshape(-1, 28*28)).reshape(-1, 28, 28)

model_selu = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.Dense(100, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.Dense(10, activation="softmax")
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


### 1.4. Batch Normalization

Batch Normalization (BN) adalah teknik yang mengatasi masalah gradien tidak stabil dengan menambahkan operasi normalisasi tepat sebelum atau sesudah fungsi aktivasi setiap lapisan tersembunyi.  Operasi ini menormalkan dan menormalisasi input setiap input (mean 0, standar deviasi 1) kemudian menskalakannya dan menggeser hasilnya menggunakan dua vektor parameter baru (satu untuk penskalaan $\gamma$, satu untuk pergeseran $\beta$).

* **Manfaat**:
    * Mengurangi masalah *vanishing/exploding gradients*.
    * Jaringan kurang sensitif terhadap inisialisasi bobot.
    * Memungkinkan penggunaan *learning rates* yang lebih besar, mempercepat pelatihan.
    * Bertindak sebagai regularizer, mengurangi kebutuhan akan teknik regularisasi lainnya.
* **Cara kerja**: Selama pelatihan, BN memperkirakan mean dan standar deviasi input berdasarkan mini-batch saat ini.  Selama inferensi, ia menggunakan rata-rata bergerak dari mean dan standar deviasi input yang dihitung selama pelatihan.

**Kode Implementasi (Batch Normalization)**

In [3]:
# Contoh model dengan Batch Normalization setelah lapisan Dense dan sebelum aktivasi
model_bn = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(), # Normalisasi input pertama
    keras.layers.Dense(300, use_bias=False), # Hapus bias karena BN akan menambahkannya
    keras.layers.BatchNormalization(),
    keras.layers.Activation("elu"),
    keras.layers.Dense(100, use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("elu"),
    keras.layers.Dense(10, activation="softmax")
])

# Contoh model dengan Batch Normalization setelah fungsi aktivasi (opsi lain)
model_bn_after_activation = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="elu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="elu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation="softmax")
])

model_bn.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_3 (Flatten)             │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 784)            │         3,136 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 300)            │       235,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 300)            │         1,200 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 300)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 100)            │        30,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 100)            │           400 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 10)             │         1,010 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 270,946 (1.03 MB)

 Trainable params: 268,578 (1.02 MB)

 Non-trainable params: 2,368 (9.25 KB)

### 1.5. Gradient Clipping

Gradient Clipping adalah teknik populer untuk mengatasi masalah *exploding gradients* dengan membatasi gradien agar tidak melebihi ambang batas tertentu selama *backpropagation*.  Ini sering digunakan pada jaringan neural berulang (RNN) di mana *Batch Normalization* sulit diterapkan.

**Kode Implementasi (Gradient Clipping)**

In [4]:
# Contoh penggunaan Gradient Clipping pada optimizer
optimizer_clipped = keras.optimizers.SGD(clipvalue=1.0) # Memotong setiap komponen gradien antara -1.0 dan 1.0
model_clipped = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])
model_clipped.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer_clipped)

# Opsi lain: clipnorm (memotong seluruh vektor gradien jika norma L2-nya melebihi ambang batas)
optimizer_clipped_norm = keras.optimizers.SGD(clipnorm=1.0)
model_clipped_norm = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])
model_clipped_norm.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer_clipped_norm)

---

## Bagian 2: Memanfaatkan Model Terlatih (Pretrained Models)

### 2.1. Reusing Pretrained Layers (Transfer Learning)

*Transfer learning* adalah teknik yang sangat efektif di mana kita menggunakan kembali lapisan bawah dari jaringan neural yang sudah terlatih pada tugas serupa.  Ini secara signifikan mempercepat pelatihan dan membutuhkan lebih sedikit data pelatihan.
* **Strategi**: Ganti lapisan output model asli, bekukan lapisan-lapisan yang digunakan kembali (membuat bobotnya tidak dapat dilatih) untuk beberapa epoch awal, dan kemudian secara opsional *unfreeze* lapisan atas dan sesuaikan *learning rate* untuk *fine-tuning*.
* **Manfaat**: Mengurangi *overfitting* karena model tidak perlu mempelajari fitur tingkat rendah dari awal.

**Kode Implementasi (Transfer Learning)**

In [5]:
# Contoh (ini adalah contoh konseptual, model_A perlu dilatih sebelumnya)
# Misalkan model_A adalah model yang sudah terlatih untuk tugas serupa
# model_A = keras.models.load_model("my_model_A.h5")

# Ambil semua lapisan kecuali lapisan output
# model_B_on_A = keras.models.Sequential(model_A.layers[:-1])
# Tambahkan lapisan output baru untuk tugas B
# model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))

# Bekukan lapisan-lapisan yang digunakan kembali
# for layer in model_B_on_A.layers[:-1]:
#     layer.trainable = False

# Kompilasi model setelah membekukan lapisan
# model_B_on_A.compile(loss="binary_crossentropy", optimizer="sgd", metrics=["accuracy"])

# Latih model untuk beberapa epoch
# history = model_B_on_A.fit(X_train_B, y_train_B, epochs=4, validation_data=(X_valid_B, y_valid_B))

# # Unfreeze lapisan-lapisan yang digunakan kembali untuk fine-tuning
# for layer in model_B_on_A.layers[:-1]:
#     layer.trainable = True

# # Sesuaikan learning rate agar lebih kecil
# optimizer_fine_tune = keras.optimizers.SGD(lr=1e-4)
# model_B_on_A.compile(loss="binary_crossentropy", optimizer=optimizer_fine_tune, metrics=["accuracy"])

# # Lanjutkan pelatihan
# history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16, validation_data=(X_valid_B, y_valid_B))

### 2.2. Unsupervised Pretraining

Jika Anda memiliki banyak data pelatihan tidak berlabel tetapi sedikit data berlabel, Anda dapat menggunakan *unsupervised pretraining*. Ini melibatkan pelatihan model tak berarah (seperti autoencoder atau GAN) pada semua data (berlabel dan tidak berlabel), kemudian menggunakan kembali lapisan bawah model tersebut sebagai dasar untuk tugas Anda yang sebenarnya (menggunakan data berlabel).

### 2.3. Pretraining on an Auxiliary Task

Jika Anda tidak memiliki banyak data berlabel, Anda bisa melatih jaringan neural pertama pada *tugas bantu* (auxiliary task) di mana Anda dapat dengan mudah memperoleh atau menghasilkan data berlabel.  Kemudian, Anda dapat menggunakan kembali lapisan-lapisan bawah jaringan tersebut untuk tugas utama Anda.

---

## Bagian 3: Optimizer yang Lebih Cepat

Pengoptimal yang lebih canggih daripada *Gradient Descent* biasa dapat secara signifikan mempercepat pelatihan model besar.

### 3.1. Momentum Optimization

Momentum Optimization mempercepat pelatihan dengan menambahkan sebagian dari pembaruan gradien sebelumnya ke pembaruan saat ini.  Ini membantu algoritma mengatasi lembah datar dan "meluncur" melewati minimum lokal lebih cepat.
* Rumus: $m \leftarrow \beta m - \eta \nabla_\theta J(\theta)$, $\theta \leftarrow \theta + m$
* $\beta$ (momentum): Biasanya 0.9.

In [8]:
optimizer_momentum = keras.optimizers.SGD(learning_rate=0.001, momentum=0.9)
# model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer_momentum, metrics=["accuracy"])

### 3.2. Nesterov Accelerated Gradient (NAG)

NAG adalah varian dari Momentum Optimization yang hampir selalu lebih cepat.  Ia mengukur gradien fungsi biaya sedikit lebih maju dalam arah momentum, menghasilkan pembaruan yang lebih akurat dan mengurangi osilasi.

In [9]:
optimizer_nag = keras.optimizers.SGD(learning_rate=0.001, momentum=0.9, nesterov=True)
# model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer_nag, metrics=["accuracy"])

### 3.3. AdaGrad

AdaGrad menyesuaikan *learning rate* untuk setiap parameter, menguranginya lebih cepat untuk dimensi yang curam.  Ini membantu mengarahkan pembaruan secara lebih langsung menuju optimum global, tetapi dapat berhenti terlalu cepat pada pelatihan jaringan neural.

In [10]:
# Tidak direkomendasikan untuk DNN yang dalam karena cenderung berhenti terlalu cepat.
optimizer_adagrad = keras.optimizers.Adagrad(learning_rate=0.001)
# model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer_adagrad, metrics=["accuracy"])

### 3.4. RMSProp

RMSProp mengatasi masalah *AdaGrad* dengan mengakumulasi hanya gradien dari iterasi terbaru menggunakan rata-rata bergerak eksponensial.  Ini mencegah *learning rate* menurun terlalu cepat.

In [11]:
optimizer_rmsprop = keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9) # rho adalah parameter peluruhan
# model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer_rmsprop, metrics=["accuracy"])

### 3.5. Adam dan Nadam Optimization

* **Adam (Adaptive Moment Estimation)**: Menggabungkan ide-ide Momentum Optimization dan RMSProp. Ia melacak rata-rata bergerak eksponensial dari gradien di masa lalu (seperti momentum) dan rata-rata bergerak eksponensial dari kuadrat gradien di masa lalu (seperti RMSProp).
* **Nadam (Nesterov Adam)**: Adam ditambah trik Nesterov Accelerated Gradient.  Seringkali konvergen sedikit lebih cepat daripada Adam.

In [12]:
optimizer_adam = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
# model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer_adam, metrics=["accuracy"])

optimizer_nadam = keras.optimizers.Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
# model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer_nadam, metrics=["accuracy"])

### 3.6. Learning Rate Scheduling

Menemukan *learning rate* yang baik sangat penting.  Memulai dengan *learning rate* yang besar dan kemudian menguranginya seiring berjalannya pelatihan dapat mempercepat konvergensi dan mencapai solusi yang lebih baik daripada dengan *learning rate* konstan yang optimal.

* **Power Scheduling**: $\eta(t) = \eta_0 / (1 + t/s)^c$. *Learning rate* menurun lebih cepat di awal dan kemudian melambat.
* **Exponential Scheduling**: $\eta(t) = \eta_0 0.1^{t/s}$. *Learning rate* menurun secara eksponensial.
* **Piecewise Constant Scheduling**: Menggunakan *learning rate* konstan untuk sejumlah epoch, lalu menurunkannya.
* **Performance Scheduling**: Mengurangi *learning rate* saat metrik validasi berhenti membaik.
* **1cycle Scheduling**: Meningkatkan *learning rate* secara linear di awal pelatihan, kemudian menguranginya secara linear, dan akhirnya menjatuhkannya lagi di akhir. Ini juga melibatkan perubahan momentum.

In [13]:
# Contoh Power Scheduling (decay parameter pada optimizer SGD)
# optimizer_power = keras.optimizers.SGD(lr=0.01, decay=1e-4)

# Contoh Exponential Scheduling dengan LearningRateScheduler callback
def exponential_decay_fn(epoch):
    return 0.01 * 0.1**(epoch / 20)

lr_scheduler = keras.callbacks.LearningRateScheduler(exponential_decay_fn)
# model.fit(X_train_scaled, y_train, epochs=30, validation_data=(X_valid_scaled, y_valid), callbacks=[lr_scheduler])

# Contoh ReduceLROnPlateau (Performance Scheduling)
lr_plateau_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)
# model.fit(X_train_scaled, y_train, epochs=30, validation_data=(X_valid_scaled, y_valid), callbacks=[lr_plateau_scheduler])

# Contoh ExponentialDecay dari keras.optimizers.schedules (learning rate per step)
# s = 20 * len(X_train) // 32 # number of steps in 20 epochs (batch size = 32)
# learning_rate_schedule = keras.optimizers.schedules.ExponentialDecay(0.01, s, 0.1)
# optimizer_schedule = keras.optimizers.SGD(learning_rate=learning_rate_schedule)
# model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer_schedule, metrics=["accuracy"])

---

## Bagian 4: Menghindari Overfitting Melalui Regularisasi

Jaringan neural yang dalam dengan jutaan parameter sangat rentan terhadap *overfitting*.  Selain *early stopping* (yang sudah dibahas di Bab 10) dan *Batch Normalization* (yang juga bertindak sebagai regularizer), ada teknik regularisasi lainnya.

### 4.1. L1 dan L2 Regularization

Sama seperti pada model linear, kita dapat menambahkan *penalty term* L1 atau L2 ke fungsi biaya untuk mengontrol kompleksitas model.
* **L2 Regularization** (Ridge): Menambahkan $\alpha \sum_{i} \theta_i^2$ ke fungsi biaya. Mendorong bobot agar tetap kecil.
* **L1 Regularization** (Lasso): Menambahkan $\alpha \sum_{i} |\theta_i|$ ke fungsi biaya. Mendorong bobot yang kurang penting menjadi nol, menghasilkan model yang *sparse* (seleksi fitur otomatis).

In [14]:
from functools import partial

# L2 regularization
RegularizedDense_L2 = partial(keras.layers.Dense,
                              activation="elu",
                              kernel_initializer="he_normal",
                              kernel_regularizer=keras.regularizers.l2(0.01))

model_l2 = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    RegularizedDense_L2(300),
    RegularizedDense_L2(100),
    RegularizedDense_L2(10, activation="softmax", kernel_initializer="glorot_uniform")
])

# L1 regularization (atau L1_L2 untuk kombinasi)
# RegularizedDense_L1 = partial(keras.layers.Dense,
#                               activation="elu",
#                               kernel_initializer="he_normal",
#                               kernel_regularizer=keras.regularizers.l1(0.01))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


### 4.2. Dropout

Dropout adalah salah satu teknik regularisasi paling populer dan sukses.  Pada setiap langkah pelatihan, setiap neuron (kecuali neuron output) memiliki probabilitas $p$ untuk "dihilangkan" sementara, artinya ia diabaikan sepenuhnya selama langkah pelatihan tersebut.
* **Dropout rate** ($p$): Biasanya antara 10% dan 50%.
* **Manfaat**: Mencegah neuron beradaptasi secara berlebihan satu sama lain, membuat jaringan lebih kuat dan lebih baik dalam menggeneralisasi.  Dapat dilihat sebagai rata-rata dari banyak jaringan neural yang lebih kecil.
* **MC Dropout**: Setelah pelatihan, membuat banyak prediksi dengan Dropout diaktifkan (mengatur `training=True`) dan merata-ratakan prediksi tersebut.  Ini memberikan estimasi ketidakpastian model dan dapat meningkatkan kinerja.

In [15]:
model_dropout = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dropout(rate=0.2), # Dropout pada input layer
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2), # Dropout setelah hidden layer pertama
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2), # Dropout setelah hidden layer kedua
    keras.layers.Dense(10, activation="softmax")
])

# Implementasi MC Dropout (untuk inferensi)
# Setelah model dilatih:
# class MCDropout(keras.layers.Dropout):
#     def call(self, inputs):
#         return super().call(inputs, training=True)

# # Buat model baru dengan lapisan MCDropout atau langsung panggil model dengan training=True
# # y_probas = np.stack([model_dropout(X_test_scaled, training=True) for sample in range(100)])
# # y_proba = y_probas.mean(axis=0)

### 4.3. Max-Norm Regularization

Max-Norm Regularization membatasi bobot $w$ dari koneksi yang masuk ke setiap neuron sehingga $||w||_2 \le r$, di mana $r$ adalah hiperparameter *max-norm*.  Ini membantu meredakan masalah gradien yang tidak stabil dan mengurangi *overfitting*.

In [16]:
model_max_norm = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal",
                        kernel_constraint=keras.constraints.max_norm(1.)), # Batasi norma bobot menjadi 1
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal",
                        kernel_constraint=keras.constraints.max_norm(1.)),
    keras.layers.Dense(10, activation="softmax")
])

---

## Bagian 5: Ringkasan dan Pedoman Praktis

**Pedoman Umum untuk Konfigurasi DNN:**

| Hiperparameter       | Nilai Default (Umum)                |
| :------------------- | :---------------------------------- |
| Kernel Initializer   | He Initialization                   |
| Activation Function  | ELU                                 |
| Normalization        | None (jika dangkal); Batch Norm (jika dalam) |
| Regularization       | Early Stopping (+ L2 reg. jika diperlukan) |
| Optimizer            | Momentum Optimization (atau RMSProp/Nadam) |
| Learning Rate Schedule | 1cycle                              |

**Pedoman untuk Jaringan yang *Self-Normalizing* (menggunakan SELU):**

| Hiperparameter       | Nilai Default (SELU)                |
| :------------------- | :---------------------------------- |
| Kernel Initializer   | LeCun Initialization                |
| Activation Function  | SELU                                |
| Normalization        | None (self-normalization)           |
| Regularization       | Alpha Dropout (jika diperlukan)    |
| Optimizer            | Momentum Optimization (atau RMSProp/Nadam) |
| Learning Rate Schedule | 1cycle                              |

**Catatan Penting:**
* Selalu normalisasi fitur input.
* Pertimbangkan *transfer learning* jika memungkinkan.
* Pertimbangkan *unsupervised pretraining* jika data berlabel terbatas.
* Jika model membutuhkan model yang *sparse*, gunakan L1 regularization.
* Untuk model *low-latency* (prediksi sangat cepat), gunakan lapisan yang lebih sedikit, gabungkan lapisan *Batch Normalization*, dan gunakan fungsi aktivasi yang lebih cepat.
* Untuk aplikasi yang sensitif terhadap risiko, gunakan MC Dropout untuk estimasi probabilitas yang lebih diandalkan dan estimasi ketidakpastian.